In [1]:
from ipywidgets import interactive
from ipywidgets import widgets
from IPython.display import display

import numpy as np
import multiprocessing
from scipy.signal import (hilbert, decimate)
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from mspacman import (PACGenerator, PhaseAmplitudeCoupling)
from mspacman.algorithm.pac_ import (mrpad, polar)
from mspacman.generator.noise import (white, pink)

In [2]:
# Simulation Parameters
# -----------------------------------------
# Select Channel
# There are 11 channels used in this data.
# Play around to see what changes. (0-10)
# -----------------------------------------
decimate_by = 1

dp = 3
da = 30
fpsize = 15
fasize = 15

p_range = (6, 50)
a_range = (60, 500)

fp = np.arange(*p_range, int(np.ceil(np.diff(p_range)/fpsize)), dtype=int)
fa = np.arange(*a_range, int(np.ceil(np.diff(a_range)/fasize)), dtype=int)

fois_lo = np.asarray([(p-dp, p+dp) for p in fp])
fois_hi = np.asarray([(a-da, a+da) for a in fa])

In [3]:
global fs, t_dur, tvec
fs = 2**14
t_dur = 8
tvec = np.arange(fs*t_dur) / fs
nsamp = fs * t_dur
binsize = 10

comod = PhaseAmplitudeCoupling(nch=1, nsamp=nsamp, freq_phase=fois_lo, freq_amp=fois_hi, sample_rate=fs,\
                               nprocs=multiprocessing.cpu_count()-1, pac='mi', nbins=10, mprocs=True)

bin_centers = np.linspace(-np.pi, np.pi, binsize+1)
def func(freq_phase, freq_amp, scale_phase, scale_amp, phase_amp, pac, noise):
    lo, hi = PACGenerator._pac_hr(fs*t_dur, pac, scale_phase, scale_amp, \
                                    freq_phase, freq_amp, fs, phase_amp=phase_amp)

    xlo = hilbert(lo)
    xhi = hilbert(hi)
    out = np.atleast_2d(xlo.real + xhi.real)
    
    if noise:
        noise_ = (white(1, nsamp, std=.5) + pink(1, nsamp, std=.5))
        out += noise_
        comod.comodulogram(out)
        xlo = np.mean(comod._xlo, axis=1)[:,np.newaxis,:]
        xhi = np.mean(comod._xhi, axis=1)[:,np.newaxis,:]    

    else:
        comod.comodulogram(out)
        xlo = decimate(xlo, 15, ftype='iir', axis=-1, zero_phase=True)[:,np.newaxis,:-1]
        xhi = decimate(xhi, 15, ftype='iir', axis=-1, zero_phase=True)[:,np.newaxis,:-1]

    pd = mrpad(np.angle(xlo), np.abs(xhi), nbins=10, axis=-1)[0,0,0,:]
    z = polar(np.angle(xlo), np.abs(xhi))[0,0,0,:]

    # Prepare Plots
    fig_pac = plt.figure(figsize=(12,6))
    gs1_pac = GridSpec(1, 1)
    gs2_pac = GridSpec(1, 3)

    gs1_pac.update(left=0.1, right=0.9, top=.9, bottom=.6, hspace=.01)
    gs2_pac.update(left=0.1, right=0.95, top=.45, bottom=.1, hspace=.01, wspace=.4)

    ax1_pac = [0, 1, 2, 3]
    ax1_pac[0] = plt.subplot(gs1_pac[:, :])
    ax1_pac[1] = plt.subplot(gs2_pac[:, 0])
    ax1_pac[2] = plt.subplot(gs2_pac[:, 1])
    ax1_pac[3] = plt.subplot(gs2_pac[:, 2])
    
    # Plot data
    ax1_pac[0].plot(tvec[:fs], out.T[:fs], c='k')
    ax1_pac[1].plot(z.real,z.imag, 'k')
    ax1_pac[1].plot([0,z.real.mean()], [0,z.imag.mean()], color='r')
    ax1_pac[2].bar(bin_centers[:-1], pd, color='k', width=.5)
    comod.plot_comodulogram(ch=0, axs=[ax1_pac[3]], cbar=False, cmap='gray')
    
    ax1_pac[1].set_xlim([-1, 1])
    ax1_pac[1].set_ylim([-1, 1])
    plt.show()

In [4]:
control_freq_phase = widgets.IntSlider(min=2, max=48, step=2, value=16)
control_freq_amp = widgets.IntSlider(min=100, max=200, step=10, value=130)
control_scale_phase = widgets.FloatSlider(min=0, max=1, step=.1, value=1)
control_scale_amp = widgets.FloatSlider(min=0, max=1, step=.1, value=.2)
control_phase_ampenv = widgets.FloatSlider(min=-np.pi, max=np.pi, step=np.pi/10, value=0)
control_pac = widgets.FloatSlider(min=0, max=1, step=.1, value=.7)

y=interactive(func,
              freq_phase=control_freq_phase,
              freq_amp=control_freq_amp,
              scale_phase=control_scale_phase,
              scale_amp=control_scale_amp,
              phase_amp=control_phase_ampenv,
              pac=control_pac,
              noise=False)

# control = VBox([control_freq_phase,
#                 control_freq_amp,
#                 control_scale_phase,
#                 control_scale_amp,
#                 control_phase_ampenv,
#                 control_pac])

display(y)

interactive(children=(IntSlider(value=16, description='freq_phase', max=48, min=2, step=2), IntSlider(value=13…

In [5]:
# comod.kill()